In [ ]:
import openai # depricated can probably remove
import random # 
import string #
import pandas as pd # 
from datetime import datetime 
import json

from scripts.fact_gen import generate_facts_k_tokens # Helper Functions
from scripts.build_prompt import build_prompt_for_all_keys #


# Log Directory (Deprecated)
# from pathlib import Path # Log/Experiment Storage
# LOG_DIR = Path("experiment_logs")
# LOG_DIR.mkdir(exist_ok=True)

In [ ]:
import os
from dotenv import load_dotenv # APIs
from openai import OpenAI
import tiktoken # Tokenizer

load_dotenv() # Get dem api key

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Model
MODEL_NAME = "gpt-3.5-turbo"
ENCODING_NAME = "gpt-3.5-turbo"

# For token counting
encoding = tiktoken.encoding_for_model(ENCODING_NAME)

In [2]:
def count_gpt_tokens(text: str) -> int:
    """Return how many tokens `text` uses under the GPT-3.5 (or GPT-4) tokenizer."""
    return len(encoding.encode(text))

def query_gpt(prompt: str, model: str = MODEL_NAME, temperature: float = 0.0) -> str:
    """
    Calls OpenAI's ChatCompletion API with the given prompt,
    returns the assistant's message content.
    """
    response = client.responses.create(
        model=model,
        instructions="You are a helpful assistant. Respond with only the correct answer on each line.",
        input=prompt,
        temperature=temperature
    )
    return response.output_text

In [30]:
# Load Token Set
filename = "data/tokens/alpha_tokens.json"
with open(filename, 'r') as f:
        token_set = set(json.load(f))

# Convert to list and count list
single_token_vocab = list(token_set)
print("Single-token vocab size:", len(single_token_vocab))

# Ensure unique tokens by count_gpt_tokens
check_good = True
for token in single_token_vocab:
    if count_gpt_tokens(token) != 1:
        print(f"Token '{token}' has {count_gpt_tokens(token)} tokens, not 1.")
        check_good = False
if check_good:
    print("All tokens are single tokens.")

Single-token vocab size: 10000
All tokens are single tokens.


In [ ]:
def example_prompt_and_response(single_token_vocab,num_facts=5, k=3): # For Debugging/testing
    """
    Example prompt and response using the given single-token vocabulary.
    Prints results to the console.
    Args:
        single_token_vocab (list): List of single-token vocabulary.
        num_facts (int): Number of facts to generate.
        k (int): Number of tokens per key-value pair.
    """
    facts_list, key_value_dict = generate_facts_k_tokens(num_facts, k, single_token_vocab) # Generate facts

    prompt, question_keys_in_order = build_prompt_for_all_keys(facts_list) # Build prompt  

    print("=== Prompt===\n")
    print(prompt)

    print("\n=== correct ===")
    for key in question_keys_in_order: # Print the correct answers in order
        print(f"{key_value_dict[key]}")

    answer = query_gpt(prompt, model=MODEL_NAME, temperature=0.0) # Query GPT
    print("\n=== GPT RESPONSE ===")
    print(answer)

### UNCOMMENT TO RUN EXAMPLE PROMPT AND RESPONSE
#example_prompt_and_response(single_token_vocab=single_token_vocab, num_facts=5, k=3)

Single-token vocab size: 10000


In [17]:
def evaluate_token_sequences(response_seqs: list[str], correct_seqs: list[str]):
    """
    Compares tokenized sequences using '|' separator.

    Returns:
        sequence_accuracy: % of full sequences that match exactly
        token_accuracy: % of individual tokens that match
    """
    total_sequences = min(len(response_seqs), len(correct_seqs))
    total_tokens = 0
    matched_tokens = 0
    matched_sequences = 0

    for resp_line, corr_line in zip(response_seqs, correct_seqs):
        resp_tokens = resp_line.strip().split("|")
        corr_tokens = corr_line.strip().split("|")

        if resp_tokens == corr_tokens:
            matched_sequences += 1

        for r, c in zip(resp_tokens, corr_tokens):
            if r == c:
                matched_tokens += 1

        total_tokens += min(len(resp_tokens), len(corr_tokens))

    sequence_accuracy = matched_sequences / total_sequences if total_sequences else 0
    token_accuracy = matched_tokens / total_tokens if total_tokens else 0

    return sequence_accuracy, token_accuracy


In [31]:
def grade_response(response_text: str, question_keys_in_order: list, key_value_dict: dict, num_facts: int, k: int):
    """
    Grades GPT response:
    -- Checks if response starts with a non-supported character
    -- Converts response to a list of strings
    -- Compares token count with expected token count
    -- Compares response with expected response
    Returns a tuple of (sequence_accuracy, token_accuracy, major_format_flaw)
    """
    # Check if response starts with a non supported character (a-z lowercase is only supported)
    major_format_flaw = False
    if not response_text[0].isalpha():
        print("Response starts with an unsupported character.")
        major_format_flaw = True

    # expected response
    correct_response_seqs = [key_value_dict[key] for key in question_keys_in_order]

    # Parse model response
    response_seqs = [line.strip() for line in response_text.strip().split("\n") if line.strip()]

    # Token Count (Using Tokenizer)
    expected_token_count = sum(count_gpt_tokens(seq) for seq in correct_response_seqs)
    response_token_count = sum(count_gpt_tokens(seq) for seq in response_seqs)

    # Evaluate
    scores = evaluate_token_sequences(response_seqs, correct_response_seqs)
    return scores, major_format_flaw, expected_token_count, response_token_count

# Query GPT FOR DEBUGGING
def grade_test(single_token_vocab, num_facts=5, k=3): 
    """
    Generates a single test case,
    queries GPT for a response, and grades the response.
    Prints the results to the console.
    Args:
        single_token_vocab (list): List of single-token vocabulary.
        num_facts (int): Number of facts to generate.
        k (int): Number of tokens per key-value pair.
    """
    facts_list, key_value_dict = generate_facts_k_tokens(num_facts, k, single_token_vocab)
    prompt, question_keys_in_order = build_prompt_for_all_keys(facts_list)
    answer = query_gpt(prompt, model=MODEL_NAME, temperature=0.0)
    (response_seqs,correct_response_seqs), major_fromat_flaw, expected_token_count, response_token_count = grade_response(answer,question_keys_in_order=question_keys_in_order,key_value_dict=key_value_dict, num_facts=num_facts,k=k)
    # Print Results
    print("=== Prompt===\n")
    print(prompt)
    print("\n=== Response ===")
    print(answer)
    print("\n=== Correct ===")
    correct_response_text = "\n".join(key_value_dict[key] for key in question_keys_in_order)
    print(correct_response_text)
    print("\n=== Accuracy ===")
    print(f"Sequence Accuracy: {response_seqs:.2f}")
    print(f"Token Accuracy: {correct_response_seqs:.2f}")
    print(f"Major Format Flaw: {major_fromat_flaw}")

# Uncomment to Run the test
#grade_test(single_token_vocab=single_token_vocab, num_facts=5, k=3)


In [ ]:
def run_experiments(
    facts_list_sizes=[3, 6],
    token_sizes=[2, 3],
    trials=1,
    output_dir="results"
):
    """Run GPT response evaluations and save structured JSON per (num_facts, k) combo."""
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    single_token_vocab = list(token_set)

    for num_facts in facts_list_sizes:
        for k in token_sizes:
            group_id = f"{num_facts}facts_{k}tokens"
            timestamp = datetime.utcnow().strftime("%Y%m%dT%H%M%S")
            file_id = f"{group_id}_{timestamp}"
            output_path = Path(output_dir) / f"{file_id}.json"

            results_group = {
                "id": file_id,
                "num_facts": num_facts,
                "k": k,
                "trials": []
            }

            for trial_idx in range(trials):
                # Generate data
                facts_list, key_value_dict = generate_facts_k_tokens(num_facts, k, single_token_vocab)
                prompt_text, question_keys = build_prompt_for_all_keys(facts_list)

                try:
                    response = query_gpt(prompt_text, model=MODEL_NAME, temperature=0.0)
                except Exception as e:
                    response = f"ERROR: {str(e)}"

                correct_response_text = "\n".join(key_value_dict[key] for key in question_keys)

                # Grade and evaluate
                (seq_acc, tok_acc), major_format_flaw, expected_token_count, response_token_count = grade_response(
                    response, question_keys, key_value_dict, num_facts, k
                )

                trial_record = {
                    "trial": trial_idx,
                    "sequence_accuracy": seq_acc,
                    "token_accuracy": tok_acc,
                    "major_format_flaw": major_format_flaw,
                    "prompt_text": prompt_text,
                    "response_text": response,
                    "response_token_count": response_token_count,
                    "expected_response_text": correct_response_text,
                    "expected_token_count": expected_token_count
                }

                results_group["trials"].append(trial_record)

            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(results_group, f, indent=2)

    return f"Saved results grouped by (num_facts, k) in: {output_dir}"


run_experiments(
    facts_list_sizes=[3, 6],
    token_sizes=[2, 3],
    trials=1,
    output_dir="results"
)


C:\Users\Marco\AppData\Local\Temp\ipykernel_4520\2723106793.py:14: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime("%Y%m%dT%H%M%S")


Expected Token Count = 6
Response Token Count = 6

=== GPT RESPONSE ===
acey|arefa
ibaba|batis
nte|rgctx

=== Correct Response ===
acey|arefa
ibaba|batis
nte|rgctx

Scores (sequence_accuracy, token_accuracy): (1.0, 1.0)
Expected Token Count = 9
Response Token Count = 9

=== GPT RESPONSE ===
uple|ancel|oria
nick|gabe|ienne
hee|nels|anal

=== Correct Response ===
uple|ancel|oria
nick|gabe|ienne
hee|nels|anal

Scores (sequence_accuracy, token_accuracy): (1.0, 1.0)
Response starts with an unsupported character.
Expected Token Count = 12
Response Token Count = 12

=== GPT RESPONSE ===
1) tes|amus
2) blade|oran
3) drink|duced
4) rcode|pond
5) frame|ocale
6) osg|pear

=== Correct Response ===
tes|amus
blade|oran
drink|duced
rcode|pond
frame|ocale
osg|pear

Scores (sequence_accuracy, token_accuracy): (0.0, 0.5)
Response starts with an unsupported character.
Expected Token Count = 18
Response Token Count = 18

=== GPT RESPONSE ===
1) oad|qing|avic
2) arily|rious|sudo
3) cript|aping|hooks
4) efu

'Saved results grouped by (num_facts, k) in: results'